In [1]:
import json
import os
import pickle

from optimized_ingestion.camera_config import camera_config
from optimized_ingestion.payload import Payload
from optimized_ingestion.pipeline import Pipeline
from optimized_ingestion.stages.decode_frame.parallel_decode_frame import ParallelDecodeFrame
from optimized_ingestion.stages.decode_frame.decode_frame import DecodeFrame
from optimized_ingestion.stages.detection_2d.yolo_detection import YoloDetection
from optimized_ingestion.stages.filter_car_facing_sideway import FilterCarFacingSideway
from optimized_ingestion.stages.detection_estimation import DetectionEstimation
from optimized_ingestion.stages.tracking_2d.strongsort import StrongSORT
from optimized_ingestion.stages.tracking_2d.tracking_2d import Tracking2D, Tracking2DResult
from optimized_ingestion.stages.tracking_3d.from_2d_and_road import From2DAndRoad
from optimized_ingestion.stages.tracking_3d.tracking_3d import Tracking3DResult
# from optimized_ingestion.trackers.yolov5_strongsort_osnet_tracker import TrackingResult
from optimized_ingestion.video import Video

YOLOv5 🚀 2022-11-10 Python-3.10.6 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12196MiB)

Using cache found in /data/chanwutk/code/apperception/weights/ultralytics_yolov5_master


Using cuda:0


YOLOv5 🚀 2022-11-22 Python-3.10.6 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12196MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [2]:
# from optimized_ingestion.cache import disable_cache
# disable_cache()

In [3]:
BOSTON_VIDEOS = [
#     "scene-0757-CAM_FRONT",
    # "scene-0103-CAM_FRONT",
    # "scene-0553-CAM_FRONT",
    # "scene-0665-CAM_FRONT",
#     "scene-0655-CAM_FRONT_RIGHT",
    "scene-0655-CAM_BACK_RIGHT",
]

NUSCENES_PROCESSED_DATA = "NUSCENES_PROCESSED_DATA"

In [4]:
class DataclassJSONEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, Tracking3DResult):
            return {
                "detection_id": o.detection_id,
                "object_id": o.object_id,
                "point_from_camera": o.point_from_camera,
                "point": o.point.tolist()
            }
        if isinstance(o, Tracking2DResult):
            return {
                "detection_id": o.detection_id,
                "frame_idx": o.frame_idx,
                "object_id": o.object_id,
                "bbox_left": o.bbox_left,
                "bbox_top": o.bbox_top,
                "bbox_w": o.bbox_w,
                "bbox_h": o.bbox_h,
                "object_type": o.object_type,
                "confidence": o.confidence
            }
        return super().default(o)

In [ ]:
pipeline = Pipeline()
pipeline.add_filter(filter=DecodeFrame())
pipeline.add_filter(filter=YoloDetection())

pipeline.add_filter(filter=DetectionEstimation())
pipeline.add_filter(filter=StrongSORT())

pipeline.add_filter(filter=From2DAndRoad())

if NUSCENES_PROCESSED_DATA in os.environ:
    DATA_DIR = os.environ[NUSCENES_PROCESSED_DATA]
else:
    DATA_DIR = "/work/apperception/data/nuScenes/full-dataset-v1.0/Mini"
with open(os.path.join(DATA_DIR, "videos/boston-seaport", "frames.pickle"), "rb") as f:
    videos = pickle.load(f)

metadata = {}
for name, video in videos.items():
#     if name not in BOSTON_VIDEOS:
#         continue
#     if not name.endswith('CAM_FRONT'):
#         continue
#     if name != 'scene-0553-CAM_FRONT_LEFT':
#         continue

    print(name, '--------------------------------------------------------------------------------')
    frames = Video(
        os.path.join(DATA_DIR, "videos/boston-seaport", video["filename"]),
        [camera_config(*f, 0) for f in video["frames"]],
        video["start"],
    )

    output = pipeline.run(Payload(frames))
    metadata[name] = From2DAndRoad.get(output)
with open(f"./outputs/trackings-with-estimation.json", "w") as f:
    json.dump(metadata, f, cls=DataclassJSONEncoder)

YOLOv5 🚀 2022-11-10 Python-3.10.6 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12196MiB)

Using cache found in /data/chanwutk/code/weights/ultralytics_yolov5_master
YOLOv5 🚀 2022-11-22 Python-3.10.6 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12196MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


scene-0103-CAM_FRONT_LEFT --------------------------------------------------------------------------------
Stage:  DecodeFrame
None
388
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Detection2D.YoloDetection
None
388
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK

 11%|█████████████████▎                                                                                                                                              | 42/387 [00:04<00:36,  9.51it/s]/data/chanwutk/code/apperception/optimized_ingestion/stages/detection_estimation/utils.py:139: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for intersect in intersection:
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 387/387 [00:26<00:00, 14.35it/s]
YOLOv5 🚀 2022-11-10 Python-3.10.6 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12196MiB)



388
None
  filtered frames: 78.6082474226804%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK..............KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK.KKKKK
KK............KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK............K............K............
K............K........KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Tracking2D.StrongSORT
Successfully loaded pretrained weights from "/data/chanwutk/code/apperception/weights/osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 388/388 [01:11<00:00,  5.39it/s]


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 388/388 [01:26<00:00,  4.46it/s]


None
388
  filtered frames: 85.05154639175258%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK..........KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KK............KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK............K....
........K............KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Tracking3D.From2DAndRoad


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 388/388 [00:00<00:00, 10167.82it/s]


None
388
  filtered frames: 85.05154639175258%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK..........KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KK............KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK............K....
........K............KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
scene-0103-CAM_BACK_RIGHT --------------------------------------------------------------------------------
Stage:  DecodeFrame
None
388
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage: 

 11%|█████████████████▎                                                                                                                                              | 42/387 [00:03<00:31, 11.05it/s]/data/chanwutk/code/apperception/optimized_ingestion/stages/detection_estimation/utils.py:139: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for intersect in intersection:
 64%|█████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                         | 248/387 [00:13<00:09, 14.66it/s]

In [ ]:

pipeline = Pipeline()
pipeline.add_filter(filter=DecodeFrame())
pipeline.add_filter(filter=YoloDetection())

# pipeline.add_filter(filter=DetectionEstimation())
pipeline.add_filter(filter=StrongSORT())

pipeline.add_filter(filter=From2DAndRoad())

if NUSCENES_PROCESSED_DATA in os.environ:
    DATA_DIR = os.environ[NUSCENES_PROCESSED_DATA]
else:
    DATA_DIR = "/work/apperception/data/nuScenes/full-dataset-v1.0/Mini"
with open(os.path.join(DATA_DIR, "videos/boston-seaport", "frames.pickle"), "rb") as f:
    videos = pickle.load(f)

metadata = {}
for name, video in videos.items():
#     if name not in BOSTON_VIDEOS:
#         continue
#     if not name.endswith('CAM_FRONT'):
#         continue
    print(name, '--------------------------------------------------------------------------------')
    frames = Video(
        os.path.join(DATA_DIR, "videos/boston-seaport", video["filename"]),
        [camera_config(*f, 0) for f in video["frames"]],
        video["start"],
    )

    output = pipeline.run(Payload(frames))
    metadata[name] = From2DAndRoad.get(output)
with open(f"./outputs/trackings-without-estimation.json", "w") as f:
    json.dump(metadata, f, cls=DataclassJSONEncoder)